# Face recognition for embedded computing

## Author:
## [Dr. Rahul Remanan](https://www.linkedin.com/in/rahulremanan)

## Import dependent libraries

In [1]:
import face_recognition
import cv2
import time
import numpy as np

## Set variables for notebook behavior

In [2]:
webcam_mode = False
videoFile_mode = True

In [3]:
video_source = './Barack_Obama_2004_DNC_Speech_CSPAN.mp4'
save_path = './proc_vid.avi'

## Load input image

In [4]:
if webcam_mode:
    video_capture = cv2.VideoCapture(0)
elif videoFile_mode:
    video_capture = cv2.VideoCapture(video_source)

In [5]:
! wget https://ichef.bbci.co.uk/news/320/cpsprodpb/E225/production/_93339875_obamalaughing.jpg -O obama.jpg

--2019-06-14 10:52:19--  https://ichef.bbci.co.uk/news/320/cpsprodpb/E225/production/_93339875_obamalaughing.jpg
Resolving ichef.bbci.co.uk (ichef.bbci.co.uk)... 23.78.209.21, 2600:141b:7000:1bc::f33, 2600:141b:7000:18b::f33
Connecting to ichef.bbci.co.uk (ichef.bbci.co.uk)|23.78.209.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7444 (7.3K) [image/jpeg]
Saving to: ‘obama.jpg’

obama.jpg           100%[===================>]   7.27K  --.-KB/s    in 0s      

2019-06-14 10:52:19 (71.4 MB/s) - ‘obama.jpg’ saved [7444/7444]



In [6]:
! wget https://thehill.com/sites/default/files/styles/thumb_small_article/public/bidenjoe_050719getty.jpg?itok=5jH57P6x -O biden.jpg

--2019-06-14 10:52:19--  https://thehill.com/sites/default/files/styles/thumb_small_article/public/bidenjoe_050719getty.jpg?itok=5jH57P6x
Resolving thehill.com (thehill.com)... 151.101.194.217, 151.101.66.217, 151.101.130.217, ...
Connecting to thehill.com (thehill.com)|151.101.194.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23944 (23K) [image/jpeg]
Saving to: ‘biden.jpg’

biden.jpg           100%[===================>]  23.38K  --.-KB/s    in 0.02s   

2019-06-14 10:52:19 (1.39 MB/s) - ‘biden.jpg’ saved [23944/23944]



## Generate face recognition embeddings

In [7]:
obama_image = face_recognition.load_image_file("obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

biden_image = face_recognition.load_image_file("biden.jpg")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

## Create arrays of face encodings and labels

In [8]:
known_face_encodings = [obama_face_encoding,
                        biden_face_encoding]

known_face_names = ["Barack Obama",
                    "Joe Biden"]

In [9]:
detection_rate = 15
num_proc_frames = None
verbose = False

In [10]:
length = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

In [11]:
fps = video_capture.get(cv2.CAP_PROP_FPS)
print ("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
img_w, img_h = int(video_capture.get(3)),int(video_capture.get(4))
print ("Source image width: "+ str(img_w))
print ("Source image height: "+ str(img_h))
fourcc = cv2.VideoWriter_fourcc(*'MJPG')#cv2.VideoWriter_fourcc(*'XVID')#
video_writer = cv2.VideoWriter(save_path, 
                               fourcc, 
                               fps, 
                               (img_w,img_h), 
                               True)

Frames per second using video.get(cv2.CAP_PROP_FPS) : 29.970050362180636
Source image width: 480
Source image height: 360


In [ ]:
start = time.time()

frame_number = 0
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    frame_number += 1

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    try:
        rgb_frame = frame[:, :, ::-1]
    except:
        break

    if frame_number % detection_rate == 0:
        # Find all the faces and face enqcodings in the frame of video
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Loop through each face in this frame of video
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

            name = "Unknown"

            # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            if verbose:
                print ("Processing frame {} / {}".format(frame_number, length))
    try:
        video_writer.write(frame)
        if verbose:
            print("Writing frame {} / {}".format(frame_number, length))
    except:
        print("Failed writing frame {} / {}".format(frame_number, length))
    if num_proc_frames != None and frame_number == num_proc_frames:
        break
        
end = time.time()
print ('Video processed in: {} seconds ...'.format(end-start))

In [ ]:
!ffmpeg -i ./Barack_Obama_2004_DNC_Speech_CSPAN.mp4 -ab 320000 -ac 2 -ar 44100 -vn ./audio.wav

In [ ]:
!ffmpeg -y -i ./output_vid.avi -i ./audio.wav -shortest -c:v copy -c:a aac -b:a 256k  ./proc_vid_audio.mp4

In [ ]:
!HandBrakeCLI -i ./proc_vid_audio.mp4 -o ./output_vid.mp4 -e x264 -q 22 -r 15 -B 64 -X 480 -O

## Multi threaded face recognition

In [ ]:
! lscpu 

In [ ]:
import face_recognition
import cv2
from multiprocessing import Process, Manager, cpu_count
import time
import numpy

In [ ]:
webcam_mode = False
videoFile_mode = True

In [ ]:
video_source = './Barack_Obama_2004_DNC_Speech_CSPAN.mp4'
save_path = './output_vid.avi'

In [ ]:
if webcam_mode:
    video_capture = cv2.VideoCapture(0)
elif videoFile_mode:
    video_capture = cv2.VideoCapture(video_source)

In [ ]:
def next_id(current_id):
    if current_id == worker_num:
        return 1
    else:
        return current_id + 1

In [ ]:
def prev_id(current_id):
    if current_id == 1:
        return worker_num
    else:
        return current_id - 1

In [ ]:
def capture(read_frame_list):
    # Get a reference to webcam #0 (the default one)
    video_capture = cv2.VideoCapture(video_source)
    # video_capture.set(3, 640)  # Width of the frames in the video stream.
    # video_capture.set(4, 480)  # Height of the frames in the video stream.
    # video_capture.set(5, 30) # Frame rate.
    print("Width: %d, Height: %d, FPS: %d" % (video_capture.get(3), video_capture.get(4), video_capture.get(5)))

    while not Global.is_exit:
        # If it's time to read a frame
        if Global.buff_num != next_id(Global.read_num):
            # Grab a single frame of video
            ret, frame = video_capture.read()
            read_frame_list[Global.buff_num] = frame
            Global.buff_num = next_id(Global.buff_num)
        else:
            time.sleep(0.01)

    # Release webcam
    video_capture.release()

In [ ]:
def process(worker_id, read_frame_list, write_frame_list):
    known_face_encodings = Global.known_face_encodings
    known_face_names = Global.known_face_names
    while not Global.is_exit:

        # Wait to read
        while Global.read_num != worker_id or Global.read_num != prev_id(Global.buff_num):
            time.sleep(0.001)

        # Delay to make the video look smoother
        #time.sleep(Global.frame_delay)

        # Read a single frame from frame list
        frame_process = read_frame_list[worker_id]

        # Expect next worker to read frame
        Global.read_num = next_id(Global.read_num)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_frame = frame_process[:, :, ::-1]
        
        Global.frame_num +=1
        if Global.frame_num % Global.sampling_rate == 0:
            # Find all the faces and face encodings in the frame of video, cost most time
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
            # Loop through each face in this frame of video
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

                name = "Unknown"

                # If a match was found in known_face_encodings, just use the first one.
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]

                # Draw a box around the face
                cv2.rectangle(frame_process, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with a name below the face
                cv2.rectangle(frame_process, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame_process, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                if Global.verbose:
                    print ('Processing frame: {} ...'.format(Global.frame_num))

        # Wait to write
        while Global.write_num != worker_id:
            time.sleep(0.01)

        # Send frame to global
        write_frame_list[worker_id] = frame_process

        # Expect next worker to write frame
        Global.write_num = next_id(Global.write_num)

In [ ]:
# Global variables
Global = Manager().Namespace()
Global.buff_num = 1
Global.read_num = 1
Global.write_num = 1
Global.frame_delay = 0
Global.is_exit = False
Global.frame_num = 0
Global.sampling_rate = 15
Global.verbose = False
read_frame_list = Manager().dict()
write_frame_list = Manager().dict()

In [ ]:
# Number of workers (subprocess use to process frames)
worker_num = 2#cpu_count()
print ('Using: {} out of: {} available CPU cores ...'.format(worker_num,
                                                             cpu_count()))

In [ ]:
# Subprocess list
p = []

# Create a subprocess to capture frames
p.append(Process(target=capture, args=(read_frame_list,)))
p[0].start()

In [ ]:
# Load a sample picture and learn how to recognize it.
obama_image = face_recognition.load_image_file("obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# Load a second sample picture and learn how to recognize it.
biden_image = face_recognition.load_image_file("biden.jpg")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

In [ ]:
# Create arrays of known face encodings and their names
Global.known_face_encodings = [
    obama_face_encoding,
    biden_face_encoding
]
Global.known_face_names = [
    "Barack Obama",
    "Joe Biden"
]

In [ ]:
# Create workers
for worker_id in range(1, worker_num + 1):
    p.append(Process(target=process, args=(worker_id, read_frame_list, write_frame_list)))
    p[worker_id].start()

In [ ]:
fps_ = video_capture.get(cv2.CAP_PROP_FPS)
print ("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps_))
img_w, img_h = int(video_capture.get(3)),int(video_capture.get(4))
print ("Source image width: "+ str(img_w))
print ("Source image height: "+ str(img_h))
fourcc = cv2.VideoWriter_fourcc(*'MJPG')#cv2.VideoWriter_fourcc(*'XVID')#
video_writer = cv2.VideoWriter(save_path, 
                               fourcc, 
                               fps_, 
                               (img_w,img_h), 
                               True)

In [ ]:
# Start to show video
last_num = 1
fps_list = []
tmp_time = time.time()
while not Global.is_exit:
    while Global.write_num != last_num:
        last_num = int(Global.write_num)

        # Calculate fps
        delay = time.time() - tmp_time
        tmp_time = time.time()
        fps_list.append(delay)
        if len(fps_list) > 5 * worker_num:
            fps_list.pop(0)
        fps = len(fps_list) / numpy.sum(fps_list)
        if Global.verbose:
            print("fps: %.2f" % fps)

        # Calculate frame delay, in order to make the video look smoother.
        # When fps is higher, should use a smaller ratio, or fps will be limited in a lower value.
        # Larger ratio can make the video look smoother, but fps will hard to become higher.
        # Smaller ratio can make fps higher, but the video looks not too smoother.
        # The ratios below are tested many times.
#         if fps < 6:
#             Global.frame_delay = (1 / fps) * 0.75
#         elif fps < 20:
#             Global.frame_delay = (1 / fps) * 0.5
#         elif fps < 30:
#             Global.frame_delay = (1 / fps) * 0.25
#         else:
#             Global.frame_delay = 0

        # Display the resulting image
        #print (write_frame_list[prev_id(Global.write_num)])
        video_writer.write(write_frame_list[prev_id(Global.write_num)])
print ('Video processed in: {} seconds ...'.format(time.time()-tmp_time))